In [71]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()
import pandas as pd
import pyterrier as pt
from tqdm import tqdm

from chatgpt import transform_query

tira = Client()

In [72]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [73]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
bm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25

In [74]:
bm25(pt_dataset.get_topics('query').head(3))

,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness
...,...,...,...,...,...,...
2995,3,112098,2015.tist_journal-ir0volumeA6A3.2,995,13.875480,social media detect self harm
2996,3,58719,2020.semeval-1.286,996,13.873003,social media detect self harm
2997,3,47623,W19-8666,997,13.869318,social media detect self harm
2998,3,82298,2010.sigirconf_conference-2010.186,998,13.869318,social media detect self harm


In [75]:
topics = pt_dataset.get_topics('query')
query = topics['query'][0]
print(query)
print(transform_query(query, prompt_technique="chain of thought"))

retrieval system improving effectiveness
CHAIN OF THOUGHT:
retrieval system improving effectiveness Improving the effectiveness of a retrieval system involves several strategies such as optimizing the indexing process, refining the search algorithms, enhancing the user interface for easier navigation, and incorporating advanced features like semantic search and machine learning algorithms 


In [76]:
for i in range(len(topics)):
    print(i, end="")
    query = topics['query'][i] #68 querys
    new_query = transform_query(query, prompt_technique="synonym")
    new_query.replace("'", " ")
    topics['query'][i] = new_query

0SYNONYM EXPANSION
1SYNONYM EXPANSION
2SYNONYM EXPANSION
3SYNONYM EXPANSION
4SYNONYM EXPANSION
5SYNONYM EXPANSION
6SYNONYM EXPANSION
7SYNONYM EXPANSION
8SYNONYM EXPANSION
9SYNONYM EXPANSION
10SYNONYM EXPANSION
11SYNONYM EXPANSION
12SYNONYM EXPANSION
13SYNONYM EXPANSION
14SYNONYM EXPANSION
15SYNONYM EXPANSION
16SYNONYM EXPANSION
17SYNONYM EXPANSION
18SYNONYM EXPANSION
19SYNONYM EXPANSION
20SYNONYM EXPANSION
21SYNONYM EXPANSION
22SYNONYM EXPANSION
23SYNONYM EXPANSION
24SYNONYM EXPANSION
25SYNONYM EXPANSION
26SYNONYM EXPANSION
27SYNONYM EXPANSION
28SYNONYM EXPANSION
29SYNONYM EXPANSION
30SYNONYM EXPANSION
31SYNONYM EXPANSION
32SYNONYM EXPANSION
33SYNONYM EXPANSION
34SYNONYM EXPANSION
35SYNONYM EXPANSION
36SYNONYM EXPANSION
37SYNONYM EXPANSION
38SYNONYM EXPANSION
39SYNONYM EXPANSION
40SYNONYM EXPANSION
41SYNONYM EXPANSION
42SYNONYM EXPANSION
43SYNONYM EXPANSION
44SYNONYM EXPANSION
45SYNONYM EXPANSION
46SYNONYM EXPANSION
47SYNONYM EXPANSION
48SYNONYM EXPANSION
49SYNONYM EXPANSION
50SYNONYM 

In [77]:

for i in range(len(topics)):
    topics['query'][i] = topics['query'][i].replace("'", " ").replace('"', ' ')
    print(topics['query'][i])

 retrieval system improving effectiveness information retrieval system enhancing efficiency search system boosting performance data retrieval system augmenting effectiveness 
 machine learning language identification artificial intelligence speech recognition computational linguistics text classification 
 social media detect self harm online platforms identify self injury social networks recognize self damage digital media spot self mutilation 
 stemming for arabic languages root extraction for Semitic languages word truncation for Arabic languages morphological analysis for Arabic languages 
 audio based animal recognition sound based creature identification acoustic based fauna identification 
 comparison different retrieval models contrast various information retrieval systems analysis diverse search models evaluation assorted retrieval methods 
 cache architecture memory design storage structure data organization 
 document scoping formula document range equation document boundary

In [78]:
run = bm25(topics)

In [79]:
persist_and_normalize_run(run, system_name='gptcot-pipeline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
